## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-21-06-48-33 +0000,startribune,Live: Country icon Willie Nelson takes the sta...,https://www.startribune.com/live-farm-aid-40-k...
1,2025-09-21-06-33-49 +0000,nypost,Virginia preschool teacher fired after she’s c...,https://nypost.com/2025/09/21/us-news/virginia...
2,2025-09-21-06-12-00 +0000,bbc,Starmer set to announce UK recognition of Pale...,https://www.bbc.com/news/articles/ce800enrglzo...
3,2025-09-21-05-58-14 +0000,nyt,They Helped Oust a Dictator. Now the New Regim...,https://www.nytimes.com/2025/09/20/world/ameri...
4,2025-09-21-05-45-07 +0000,nypost,"Michigan student, 11, expelled after taking gu...",https://nypost.com/2025/09/21/us-news/michigan...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,30
152,kirk,14
151,charlie,12
28,new,10
175,visa,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
85,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...,73
170,2025-09-20-10-00-00 +0000,wapo,"Under Trump, Palestinians seeking U.S. asylum ...",https://www.washingtonpost.com/world/2025/09/2...,62
50,2025-09-20-23-11-48 +0000,cbc,White House scrambles to clarify Trump plan to...,https://www.cbc.ca/news/world/trump-travel-ban...,62
172,2025-09-20-09-30-00 +0000,wsj,President Trump’s speech to the U.N. General A...,https://www.wsj.com/politics/policy/trumps-u-n...,62
83,2025-09-20-20-04-18 +0000,nypost,White House tries easing panic over high-skill...,https://nypost.com/2025/09/20/us-news/white-ho...,60


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
85,73,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...
83,48,2025-09-20-20-04-18 +0000,nypost,White House tries easing panic over high-skill...,https://nypost.com/2025/09/20/us-news/white-ho...
34,45,2025-09-21-00-48-31 +0000,nyt,Arrest in Arizona Before Charlie Kirk Memorial...,https://www.nytimes.com/2025/09/20/us/charlie-...
172,32,2025-09-20-09-30-00 +0000,wsj,President Trump’s speech to the U.N. General A...,https://www.wsj.com/politics/policy/trumps-u-n...
68,32,2025-09-20-21-33-56 +0000,nypost,High-speed police chase ends in deadly shootou...,https://nypost.com/2025/09/20/us-news/massive-...
127,29,2025-09-20-15-22-28 +0000,nypost,Q&A: NYC Mayor Eric Adams calls Jews his secre...,https://nypost.com/2025/09/20/us-news/q-amp-a-...
184,29,2025-09-20-08-21-49 +0000,nypost,"Russia launches large-scale attack on Ukraine,...",https://nypost.com/2025/09/20/world-news/russi...
19,25,2025-09-21-02-03-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
130,24,2025-09-20-14-54-49 +0000,nypost,Trump says he axed US attorney investigating N...,https://nypost.com/2025/09/20/us-news/trump-sa...
43,23,2025-09-20-23-52-13 +0000,nypost,Newsom signs anti-ICE bill banning federal law...,https://nypost.com/2025/09/20/us-news/californ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
